Poincare Feature Extraction and Visualization

In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from scipy.io import savemat, loadmat
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
import neurokit2 as nk

directory = r'S:\Sedation_monitoring\Sickbay_extract\Extract_250Hz'

In [6]:
def plot_hrv_correlations(df, title):
    # Calculate correlations with SBS
    correlations = df.corr()['SBS'].drop(['SBS', 'Patient'])
    
    # Create a DataFrame for plotting
    plot_data = pd.DataFrame({'Feature': correlations.index, 'Correlation': correlations.values})
    
    # Sort by absolute correlation value for better visualization
    plot_data = plot_data.sort_values('Correlation', key=abs, ascending=False)
    
    # Set up the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.set(style="whitegrid")
    
    # Create the bar plot
    sns.barplot(x='Feature', y='Correlation', data=plot_data, palette='coolwarm', ax=ax)
    
    # Customize the plot
    ax.set_title(title, fontsize=16)
    ax.set_xlabel('HRV Features', fontsize=12)
    ax.set_ylabel('Correlation Coefficient', fontsize=12)
    plt.xticks(rotation=90, ha='right')
    
    # Add correlation values on top of each bar
    for i, v in enumerate(plot_data['Correlation']):
        ax.text(i, v, f'{v:.2f}', ha='center', va='bottom' if v > 0 else 'top', fontsize=8)
    
    # Add a horizontal line at y=0
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    # Adjust layout
    plt.tight_layout()
    
    return fig  # Return the figure object instead of showing it

In [7]:
import numpy as np
import neurokit2 as nk
import pandas as pd
from scipy.io import loadmat
from openpyxl import load_workbook
import os

patient_info_add = r'S:\Sedation_monitoring\Sickbay_extract\Extract_250Hz\Full_Patient_List_CCDA.xlsx'
patient_info_excel = load_workbook(patient_info_add)
patient_info = patient_info_excel['Sheet1']

print('Patient List Loaded')

all_hrv_data_l1 = []
all_hrv_data_l2 = []
all_hrv_data_l3 = []

for cell_a, cell_b in zip(patient_info['A'][1:], patient_info['B'][1:]):
    patient_num = cell_a.value
    patient_mrn = cell_b.value

    patient_directory = os.path.join(directory, str(patient_mrn) + '_Study57_Tag123_EventList')
    print(f"Processing patient: {patient_num}")
    patient_data_dir = os.path.join(patient_directory, f'Patient{patient_num}_10MIN_5MIN_ECG_SBSFinal')
    
    try:
        ecg_data = loadmat(patient_data_dir)

        # Load Data
        SBS = ecg_data['sbs_score'].flatten()
        ecg1 = ecg_data['ecg1']
        ecg2 = ecg_data['ecg2']
        ecg3 = ecg_data['ecg3']

        for i in range(len(SBS)):
            try:
                ecg1_index = ecg1[0][i].flatten()
                ecg2_index = ecg2[0][i].flatten()
                ecg3_index = ecg3[0][i].flatten()

                # Check if the data is valid
                if len(ecg1_index) == 0 or np.isnan(ecg1_index).any():
                    print(f"Invalid ECG data for patient {patient_num}, index {i}")
                    continue

                total_time_seconds = 15 * 60  # 15 minutes in seconds
                num_data_points = len(ecg1_index)
                sampling_rate = num_data_points / total_time_seconds

                # Process ECG signals
                l1_signal, l1_info = nk.ecg_process(ecg1_index, sampling_rate=sampling_rate)
                l2_signal, l2_info = nk.ecg_process(ecg2_index, sampling_rate=sampling_rate)
                l3_signal, l3_info = nk.ecg_process(ecg3_index, sampling_rate=sampling_rate)

                # Calculate HRV indices
                l1_peaks, _ = nk.ecg_peaks(l1_signal["ECG_Clean"], sampling_rate=sampling_rate, correct_artifacts=True)
                l1_hrv_indices = nk.hrv(l1_peaks, sampling_rate=sampling_rate, show=False)

                l2_peaks, _ = nk.ecg_peaks(l2_signal["ECG_Clean"], sampling_rate=sampling_rate, correct_artifacts=True)
                l2_hrv_indices = nk.hrv(l2_peaks, sampling_rate=sampling_rate, show=False)

                l3_peaks, _ = nk.ecg_peaks(l3_signal["ECG_Clean"], sampling_rate=sampling_rate, correct_artifacts=True)
                l3_hrv_indices = nk.hrv(l3_peaks, sampling_rate=sampling_rate, show=False)

                # Add SBS score and patient number to HRV indices
                for hrv_indices in [l1_hrv_indices, l2_hrv_indices, l3_hrv_indices]:
                    hrv_indices['SBS'] = SBS[i]
                    hrv_indices['Patient'] = patient_num

                all_hrv_data_l1.append(l1_hrv_indices)
                all_hrv_data_l2.append(l2_hrv_indices)
                all_hrv_data_l3.append(l3_hrv_indices)

                # print(all_hrv_data_l1)

            except Exception as e:
                print(f"Error processing ECG data for patient {patient_num}, index {i}: {str(e)}")

    except Exception as e:
        print(f"Error processing patient {patient_num}: {str(e)}")

    # Combine all HRV data into DataFrames
    combined_hrv_df_l1 = pd.concat(all_hrv_data_l1, ignore_index=True)
    combined_hrv_df_l2 = pd.concat(all_hrv_data_l2, ignore_index=True)
    combined_hrv_df_l3 = pd.concat(all_hrv_data_l3, ignore_index=True)

    # print("Combined HRV DataFrames created")
    # print("L1 DataFrame shape:", combined_hrv_df_l1.shape)
    # print("L2 DataFrame shape:", combined_hrv_df_l2.shape)
    # print("L3 DataFrame shape:", combined_hrv_df_l3.shape)

    # Plot and save for Lead 1
    save_path = patient_directory

    fig1 = plot_hrv_correlations(combined_hrv_df_l1, 'Lead 1: Correlation of HRV with SBS Score')
    fig1.savefig(os.path.join(save_path, 'lead1_hrv_correlations.png'), dpi=300, bbox_inches='tight')
    plt.close(fig1)

    # Plot and save for Lead 2
    fig2 = plot_hrv_correlations(combined_hrv_df_l2, 'Lead 2: Correlation of HRV with SBS Score')
    fig2.savefig(os.path.join(save_path, 'lead2_hrv_correlations.png'), dpi=300, bbox_inches='tight')
    plt.close(fig2)

    # Plot and save for Lead 3
    fig3 = plot_hrv_correlations(combined_hrv_df_l3, 'Lead 3: Correlation of HRV with SBS Score')
    fig3.savefig(os.path.join(save_path, 'lead3_hrv_correlations.png'), dpi=300, bbox_inches='tight')
    plt.close(fig3)

    print(f"Plots saved in: {save_path}")

Patient List Loaded
Processing patient: 4


C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
C:

Invalid ECG data for patient 4, index 17
Invalid ECG data for patient 4, index 18


C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
C:

Error processing ECG data for patient 4, index 51: integer division or modulo by zero


...

Plots saved in: S:\Sedation_monitoring\Sickbay_extract\Extract_250Hz\JH58706132_Study57_Tag123_EventList
Processing patient: 13
Invalid ECG data for patient 13, index 0
Invalid ECG data for patient 13, index 1
Invalid ECG data for patient 13, index 2
Invalid ECG data for patient 13, index 3
Invalid ECG data for patient 13, index 4
Invalid ECG data for patient 13, index 5
Invalid ECG data for patient 13, index 6
Invalid ECG data for patient 13, index 7
Invalid ECG data for patient 13, index 8
Invalid ECG data for patient 13, index 9
Invalid ECG data for patient 13, index 10
Invalid ECG data for patient 13, index 11
Invalid ECG data for patient 13, index 12
Invalid ECG data for patient 13, index 13
Invalid ECG data for patient 13, index 14
Invalid ECG data for patient 13, index 15
Invalid ECG data for patient 13, index 16
Invalid ECG data for patient 13, index 17
Invalid ECG data for patient 13, index 18
Invalid ECG data for patient 13, index 19
Invalid ECG data for patient 13, index 20


C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
C:\Users\sraghav9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neurokit2\signal\signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
C:

Plots saved in: S:\Sedation_monitoring\Sickbay_extract\Extract_250Hz\JH98903105_Study57_Tag123_EventList
Processing patient: 15


C:\Users\sraghav9\AppData\Local\Temp\ipykernel_3236\467691143.py:16: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Feature', y='Correlation', data=plot_data, palette='coolwarm', ax=ax)
posx and posy should be finite values
posx and posy should be finite values
C:\Users\sraghav9\AppData\Local\Temp\ipykernel_3236\467691143.py:16: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Feature', y='Correlation', data=plot_data, palette='coolwarm', ax=ax)
posx and posy should be finite values
posx and posy should be finite values
C:\Users\sraghav9\AppData\Local\Temp\ipykernel_3236\467691143.py:16: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14

Plots saved in: S:\Sedation_monitoring\Sickbay_extract\Extract_250Hz\JH28997709_Study57_Tag123_EventList
